## Stability

In [1]:
# Set root directory of the project as the current working directory
import os
initial_dir = os.getcwd()  # Save initial directory (notebooks/)
os.chdir('..')  # Move to project/

In [2]:
# Other imports
import numpy as np
import shap

import logging
logging.basicConfig(level=logging.WARNING)

# Set the random seed for reproducibility
np.random.seed(42)

# Import Config from config.defaults and load_preprocessed_data, train_lstm_model, train_cnn_model from src.models
from config.defaults import Config
from src.models import load_saved_model, load_preprocessed_data
from src.stability import (calculate_relative_input_stability, 
                            calculate_relative_representation_stability, 
                            calculate_relative_output_stability)
from src.evaluation import evaluate_regression_model
# Load the default configuration
config = Config()
model_task = 'lstm_regression'

# Load preprocessed data
X_train, X_val, X_test, y_train_reg, y_val_reg, y_test_reg, metadata = load_preprocessed_data(model_task = model_task, eol_capacity=config.eol_capacity)

# Load saved LSTM model
lstm_model = load_saved_model(model_task, config)

def predict_wrapper(X):
    print("Input shape to model:", X.reshape(-1, X.shape[1], 1).shape)
    return model.predict(X.reshape(-1, X.shape[1], 1), verbose=0)

In [3]:
evaluate_regression_model(lstm_model, X_test, y_test_reg, metadata["y_max"])

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - mae: 0.0282  


(0.001062121824361384, 26.86453514391809)

In [4]:
lstm_model.summary()
print("Layer names:", [layer.name for layer in lstm_model.layers])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 120, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         4,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,693 (57.40 KB)

 Trainable params: 4,897 (19.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,796 (38.27 KB)

Layer names: ['input_layer', 'lstm', 'dropout', 'dense', 'dense_1']


In [5]:
# Sample 50 random sequences from the train set
X_train_2d = X_train.reshape(X_train.shape[0], -1)
X_background = shap.sample(X_train_2d, 50)

# Reshape the test set to 2D
X_test_2d = X_test.reshape(X_test.shape[0], -1)

# extract one sequence from the test set
test_instance = shap.sample(X_test_2d, 1)

In [6]:
# RIS
ris_max, ris_mean = calculate_relative_input_stability(
    lstm_model, X_background, test_instance, noise_scale=0.1
)
print(f"RIS - Max: {ris_max:.4f}, Mean: {ris_mean:.4f}")

Input diffs: [1.14018086 1.03121432 1.07843374 1.05036322 1.28662305]
Value diffs: [0.49739055 0.25368112 0.37438845 0.42267595 0.37607198]
Ratios: [0.43623829 0.24600232 0.34715944 0.40240932 0.29229383]
RIS - Max: 0.4516, Mean: 0.3211


In [7]:
# RRS
rrs_max, rrs_mean = calculate_relative_representation_stability(
    lstm_model, test_instance, noise_scale=0.1, layer_name="lstm"
)
print(f"RRS - Max: {rrs_max:.4f}, Mean: {rrs_mean:.4f}")

Input diffs: [1.23494762 1.05092483 1.07971006 1.03236545 1.04533978]
Value diffs: [0.10988549 0.07036665 0.09315553 0.09192909 0.19461286]
Ratios: [0.08897988 0.06695688 0.08627829 0.08904705 0.18617187]
RRS - Max: 0.2891, Mean: 0.1362


In [8]:
# ROS
ros_max, ros_mean = calculate_relative_output_stability(
    lstm_model, test_instance, noise_scale=0.01
)
print(f"ROS - Max: {ros_max:.4f}, Mean: {ros_mean:.4f}")

Input diffs: [0.09989139 0.10856972 0.0994006  0.10499885 0.10906094]
Value diffs: [0.01695365 0.01668036 0.07643729 0.02333772 0.03272736]
Ratios: [0.16972081 0.15363731 0.76898222 0.22226646 0.30008324]
ROS - Max: 2.0407, Mean: 0.4447


## Global

In [9]:
global_test_instance = shap.sample(X_test_2d, 10) # 10 random samples from the test set

In [10]:
# Global RIS
ris_values = []
for i in range(global_test_instance.shape[0]):
    ris_max, ris_mean = calculate_relative_input_stability(
        lstm_model, X_background, global_test_instance[i:i+1], n_perturbations=20, noise_scale=0.1
    )
    ris_values.append(ris_mean)  # Collect mean RIS for each instance
global_ris_mean = np.mean(ris_values)
global_ris_max = np.max(ris_values)
print(f"Global RIS - Mean: {global_ris_mean:.4f}, Max: {global_ris_max:.4f}")

Input diffs: [1.11767955 1.24502701 1.02886701 1.20116064 1.04880621]
Value diffs: [0.31083553 0.22381322 0.47061802 0.26186448 0.30279894]
Ratios: [0.27810791 0.17976575 0.45741384 0.21800954 0.28870818]
Input diffs: [1.11777749 1.10828294 1.16733132 1.04792744 1.09437262]
Value diffs: [0.10456605 0.09438985 0.22693103 0.18491826 0.12858903]
Ratios: [0.09354818 0.08516765 0.19440156 0.17646094 0.11750022]
Input diffs: [0.99766896 1.08094892 1.06710654 1.10091836 1.11605771]
Value diffs: [0.09108575 0.15350798 0.09309477 0.14985041 0.32953334]
Ratios: [0.09129857 0.14201224 0.08724037 0.13611401 0.2952655 ]
Input diffs: [1.12942857 1.18075168 1.09263984 1.05423001 1.11991316]
Value diffs: [0.37271581 0.45187153 0.32571636 0.37023925 0.25508554]
Ratios: [0.33000388 0.38269819 0.29810039 0.351194   0.22777261]
Input diffs: [0.94595078 1.10491597 0.98976284 0.98137581 1.22082665]
Value diffs: [0.46821974 0.49432986 0.49515209 0.49309316 0.46554443]
Ratios: [0.49497262 0.44739136 0.5002734

In [11]:
# Global RRS
rrs_values = []
for i in range(global_test_instance.shape[0]):
    rrs_max, rrs_mean = calculate_relative_representation_stability(
        lstm_model, global_test_instance[i:i+1], noise_scale=0.1, layer_name="lstm"
    )
    rrs_values.append(rrs_mean)  # Collect mean RRS for each instance
global_rrs_mean = np.mean(rrs_values)
global_rrs_max = np.max(rrs_values)
print(f"Global RRS - Mean: {global_rrs_mean:.4f}, Max: {global_rrs_max:.4f}")

Input diffs: [1.1887315  1.14681767 1.21390049 1.20294554 1.01258612]
Value diffs: [0.10120568 0.05639018 0.17637578 0.12560715 0.47241464]
Ratios: [0.08513755 0.04917101 0.14529673 0.10441632 0.46654268]
Input diffs: [1.09786653 1.17413946 1.06485412 1.03716282 1.04448509]
Value diffs: [0.02864805 0.04491398 0.02330704 0.02604211 0.02043968]
Ratios: [0.0260943  0.03825268 0.02188754 0.02510899 0.01956915]
Input diffs: [1.17540547 0.95220878 1.16206093 1.07134558 1.06705923]
Value diffs: [0.03077552 0.04177838 0.04610085 0.07303858 0.03809875]
Ratios: [0.0261829  0.04387523 0.03967163 0.06817462 0.03570443]
Input diffs: [0.9945987  1.06927491 1.19346891 1.08153827 1.07396728]
Value diffs: [0.26342827 0.17491704 0.38065392 0.46122858 0.06716336]
Ratios: [0.26485885 0.16358472 0.31894749 0.42645609 0.06253763]
Input diffs: [1.20457784 1.06748922 0.99623858 1.09195186 1.07548679]
Value diffs: [0.22181326 0.24565317 0.4312835  0.45031872 0.54491192]
Ratios: [0.18414191 0.23012239 0.4329118

In [12]:
# Global ROS
ros_values = []
for i in range(global_test_instance.shape[0]):
    ros_max, ros_mean = calculate_relative_output_stability(
        lstm_model, global_test_instance[i:i+1], n_perturbations=20, noise_scale=0.01
    )
    ros_values.append(ros_mean)  # Collect mean ROS for each instance
global_ros_mean = np.mean(ros_values)
global_ros_max = np.max(ros_values)
print(f"Global ROS - Mean: {global_ros_mean:.4f}, Max: {global_ros_max:.4f}")

Input diffs: [0.11837491 0.11617759 0.09744951 0.11172301 0.10100911]
Value diffs: [0.07466733 0.01750356 0.01493222 0.02530938 0.00204003]
Ratios: [0.63076995 0.1506621  0.15323027 0.2265369  0.02019648]
Input diffs: [0.1210551  0.1082736  0.10398619 0.10648161 0.10129266]
Value diffs: [0.00772291 0.00267419 0.00739193 0.00156619 0.00015458]
Ratios: [0.06379669 0.02469847 0.07108569 0.01470852 0.00152612]
Input diffs: [0.10111916 0.10732325 0.10808296 0.10806764 0.1123309 ]
Value diffs: [0.01506421 0.01256183 0.00065097 0.01073611 0.01326421]
Ratios: [0.14897484 0.11704666 0.00602289 0.09934618 0.11808157]
Input diffs: [0.10606648 0.11126982 0.1085583  0.10931437 0.09207107]
Value diffs: [0.05122739 0.09804016 0.01185226 0.0529846  0.09439963]
Ratios: [0.48297438 0.88110291 0.10917879 0.48469925 1.02529091]
Input diffs: [0.09717911 0.11853475 0.10391157 0.1119242  0.11779533]
Value diffs: [0.0449056  0.05144829 0.02839941 0.00866604 0.00128084]
Ratios: [0.46209111 0.43403546 0.2733036